# 🧠 Proyecto de IA - Generación de Reportes Clínicos

Este proyecto tiene como objetivo generar reportes clínicos automáticos a partir de perfiles de pacientes utilizando modelos de lenguaje como Transformers. Se incluyen procesos de carga, limpieza, análisis y generación automática de texto, así como exportación de resultados en PDF.

## 📦 1. Instalación de librerías necesarias
A continuación se instalan las librerías necesarias para la ejecución del proyecto.

In [1]:
pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 9.7 MB/s eta 0:00:00


In [2]:
!pip install fpdf2 matplotlib seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.7/245.7 kB 7.9 MB/s eta 0:00:00


In [3]:
!apt-get install -y fonts-dejavu

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra
The following NEW packages will be installed:
  fonts-dejavu fonts-dejavu-core fonts-dejavu-extra
0 upgraded, 3 newly installed, 0 to remove and 34 not upgraded.
Need to get 3,085 kB of archives.
After this operation, 10.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-dejavu-core all 2.37-2build1 [1,041 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-dejavu-extra all 2.37-2build1 [2,041 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-dejavu all 2.37-2build1 [3,192 B]
Fetched 3,085 kB in 1s (3,139 kB/s)
Selecting previously unselected package fonts-dejavu-core.
(Reading database ... 126333 files and directories currently installed.)
Preparing to unpack .../fonts-dejavu-core_2.37-2build1_all.deb ...
Unpack

## 🔗 2. Conexión con Google Drive
Se monta el Drive para poder acceder a archivos almacenados en la nube.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 🧰 3. Carga de librerías
Se importan las librerías necesarias para el procesamiento de texto, manejo de datos y visualización.

In [5]:
from transformers import pipeline
from typing import Dict, List, Optional
from datetime import datetime, timedelta
import zipfile
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import re

##📁 4. Carga de modelos entrenados
Se cargan los modelos de lenguaje previamente entrenados para generación y explicación médica automática. Esto es solo para los modelos de generación de texto guardado en Google Drive y ejecutado en Google Colab.

En caso de no contar con los datos para uso del proyecto, se pueden descargar de los siguientes enlaces:

* **Datos COVID19**: https://drive.google.com/file/d/12WOYasR0GFm_2qGprHfiuj4_3m2iBhye/view?usp=drive_link

* **Descriptores de Datos COVID19 (diccionario)**: https://drive.google.com/file/d/1hBZL6mHe_nIji1ouM9DSJXE7jIIf3maV/view?usp=drive_link

* **biogpt_explainer_model_final**: https://drive.google.com/file/d/1fyPOO7UTM6bqlawqqW_0gO5qLhhRWzaH/view?usp=drive_link

* **clean_biogpt_comorbidity_model_final.zip**: https://drive.google.com/file/d/10wnUqozpOGurjeSPDWt0Vw7LznYnYw7p/view?usp=drive_link


In [6]:
import zipfile

zipobj = zipfile.ZipFile('/content/drive/MyDrive/biogpt_explainer_model_final.zip', 'r')
zipobj.extractall(".")

In [7]:
zipobj = zipfile.ZipFile('/content/drive/MyDrive/clean_biogpt_comorbidity_model_final.zip', 'r')
zipobj.extractall(".")

##🏥 5. Definición de clases para procesamiento clínico
Se definen varias clases para la carga de datos clínicos, diccionarios médicos y clasificación con Naive Bayes:

*   COVIDDataLoader
*   DictionaryLoader
*   OptimizedNaiveBayesClassifier
*   BioExplanationTranslator

In [8]:
class COVIDDataLoader:
    def __init__(self, covid_zip_path: str):
        """
        Inicializa el cargador de datos COVID-19

        Args:
            covid_zip_path (str): Ruta al archivo ZIP que contiene los datos CSV por año
        """
        self.zip_path = covid_zip_path
        self.available_years = self._get_available_years()

    def _get_available_years(self) -> List[str]:
        """Obtiene los años disponibles en el archivo ZIP"""
        try:
            with zipfile.ZipFile(self.zip_path, 'r') as z:
                files = [f for f in z.namelist() if f.startswith('COVID19MEXICO') and f.endswith('.csv')]
                years = [f.replace('COVID19MEXICO', '').replace('.csv', '') for f in files]
                return sorted(years)
        except FileNotFoundError:
            raise FileNotFoundError(f"No se encontró el archivo ZIP en la ruta: {self.zip_path}")
        except Exception as e:
            raise RuntimeError(f"Error al leer el archivo ZIP: {str(e)}")

    def load_data_by_date_range(self, start_date: str, end_date: str, columns: Optional[List[str]] = None) -> pd.DataFrame:
        """
        Carga solo los datos dentro del rango de fechas especificado

        Args:
            start_date (str): Fecha de inicio en formato 'YYYY-MM-DD'
            end_date (str): Fecha de fin en formato 'YYYY-MM-DD'
            columns (List[str], optional): Columnas a cargar. Si es None, carga todas.

        Returns:
            pd.DataFrame: Datos filtrados por el rango de fechas
        """
        try:
            start_dt = datetime.strptime(start_date, '%Y-%m-%d')
            end_dt = datetime.strptime(end_date, '%Y-%m-%d')

            if start_dt > end_dt:
                raise ValueError("La fecha de inicio no puede ser mayor que la fecha de fin")

            # Determinar qué años necesitamos cargar
            years_to_load = set()
            for year in range(start_dt.year, end_dt.year + 1):
                if str(year) in self.available_years:
                    years_to_load.add(str(year))

            if not years_to_load:
                raise ValueError(f"No hay datos disponibles para el rango de años solicitado ({start_dt.year}-{end_dt.year})")

            print(f"\nCargando datos para los años: {', '.join(sorted(years_to_load))}")

            dfs = []
            with zipfile.ZipFile(self.zip_path, 'r') as z:
                for year in sorted(years_to_load):
                    csv_name = f'COVID19MEXICO{year}.csv'
                    print(f"Procesando {csv_name}...")

                    # Leer solo las columnas necesarias si se especificaron
                    usecols = columns if columns else None
                    df = pd.read_csv(z.open(csv_name), usecols=usecols)

                    # Convertir todas las columnas de fecha
                    df = self._convert_date_columns(df)

                    # Identificar columna de fecha automáticamente
                    date_col = self._detect_date_column(df)
                    if not date_col:
                        print(f"  - Advertencia: No se encontró columna de fecha en {csv_name}")
                        continue

                    # Convertir y filtrar por fecha
                    df['FECHA_FILTRO'] = pd.to_datetime(df[date_col], errors='coerce')
                    df = df.dropna(subset=['FECHA_FILTRO'])

                    mask = (df['FECHA_FILTRO'] >= start_dt) & (df['FECHA_FILTRO'] <= end_dt)
                    filtered_df = df[mask].copy()

                    if not filtered_df.empty:
                        dfs.append(filtered_df)
                        print(f"  - Añadidos {len(filtered_df)} registros (de {len(df)} totales)")
                    else:
                        print(f"  - No hay registros en el rango {start_date} a {end_date}")

            if not dfs:
                return pd.DataFrame()

            return pd.concat(dfs, ignore_index=True)

        except Exception as e:
            raise RuntimeError(f"Error al cargar datos: {str(e)}")

    def _detect_date_column(self, df: pd.DataFrame) -> Optional[str]:
        """Intenta identificar automáticamente la columna de fecha"""
        date_columns = ['FECHA_SINTOMAS', 'FECHA_INGRESO', 'FECHA_DEF']
        for col in date_columns:
            if col in df.columns:
                return col
        return None

    def _convert_date_columns(self, df: pd.DataFrame) -> pd.DataFrame:
      """
      Detecta y convierte automáticamente todas las columnas de fecha al tipo datetime.

      Args:
          df (pd.DataFrame): DataFrame que contiene los datos COVID-19

      Returns:
          pd.DataFrame: DataFrame con las columnas de fecha convertidas
      """
      # Lista de posibles columnas de fecha en los datos COVID-19
      date_columns = ['FECHA_ACTUALIZACION', 'FECHA_SINTOMAS',
                     'FECHA_INGRESO', 'FECHA_DEF', 'FECHA_INGRESO_ICU']

      for col in date_columns:
          if col in df.columns:
              try:
                  # Convertir a datetime y normalizar (eliminar parte horaria si no existe)
                  df[col] = pd.to_datetime(df[col], errors='coerce', format='%Y-%m-%d')
                  # Opcional: eliminar horas si todas son 00:00:00 para ahorrar espacio
                  if df[col].dt.time.nunique() == 1 and df[col].dt.time.iloc[0] == pd.Timestamp('00:00:00').time():
                      df[col] = df[col].dt.date
              except Exception as e:
                  print(f"Advertencia: No se pudo convertir la columna {col} a fecha: {str(e)}")

      return df

In [9]:
class DictionaryLoader:
    def __init__(self, dict_zip_path: str):
        """
        Inicializa el cargador de diccionarios

        Args:
            dict_zip_path (str): Ruta al archivo ZIP que contiene los diccionarios
        """
        self.zip_path = dict_zip_path

    def load_dictionaries(self) -> tuple:
        """
        Carga los diccionarios desde el archivo ZIP

        Returns:
            tuple: (diccionario_general, diccionario_alreves, descriptores)
        """
        try:
            # Diccionarios para almacenar los datos
            diccionario_general = {}
            diccionario_alreves = {}
            descriptores = {}

            with zipfile.ZipFile(self.zip_path, 'r') as zip_ref:
                # Extraer a directorio temporal
                temp_dir = '/tmp/diccionario_covid'
                if not os.path.exists(temp_dir):
                    os.makedirs(temp_dir)
                zip_ref.extractall(temp_dir)

                # Buscar archivos necesarios
                catalog_path = None
                descriptors_path = None

                for root, _, files in os.walk(temp_dir):
                    for file in files:
                        if 'catalogos' in file.lower() and file.endswith('.xlsx'):
                            catalog_path = os.path.join(root, file)
                        elif 'descriptores' in file.lower() and file.endswith('.xlsx'):
                            descriptors_path = os.path.join(root, file)

                if not catalog_path:
                    raise FileNotFoundError("No se encontró el archivo de catálogos en el ZIP")
                if not descriptors_path:
                    raise FileNotFoundError("No se encontró el archivo de descriptores en el ZIP")

                # Cargar descriptores
                descriptores = pd.read_excel(descriptors_path)
                descriptores = descriptores.set_index('NOMBRE DE VARIABLE')['FORMATO O FUENTE'].str.strip().to_dict()

                # Cargar catálogos
                excel_file = pd.ExcelFile(catalog_path)
                hojas = [h for h in excel_file.sheet_names if not h.startswith('~')]  # Excluir hojas temporales

                for hoja in hojas:
                    try:
                        sheet = excel_file.parse(hoja)

                        # Catálogo normal (CLAVE -> DESCRIPCIÓN)
                        if 'CLAVE' in sheet.columns and 'DESCRIPCIÓN' in sheet.columns:
                            diccionario_general[hoja] = sheet.set_index('CLAVE')['DESCRIPCIÓN'].str.strip().to_dict()

                        # Catálogo inverso (DESCRIPCIÓN -> CLAVE)
                        if 'DESCRIPCIÓN' in sheet.columns and 'CLAVE' in sheet.columns:
                            diccionario_crudo = sheet.set_index('DESCRIPCIÓN')['CLAVE'].to_dict()
                            diccionario_alreves[hoja] = {str(k).strip(): v for k, v in diccionario_crudo.items()}

                        # Procesar catálogo especial de municipios
                        if 'MUNICIPIOS' in hoja.upper():
                            if 'CLAVE_ENTIDAD' in sheet.columns and 'CLAVE_MUNICIPIO' in sheet.columns:
                                sheet["RESIDENCIA"] = '[' + sheet['CLAVE_ENTIDAD'].astype(str) + "," + \
                                                    sheet["CLAVE_MUNICIPIO"].astype(str) + ']'
                                diccionario_general[hoja] = sheet.set_index('RESIDENCIA')['MUNICIPIO'].to_dict()
                                diccionario_alreves[hoja] = sheet.set_index('MUNICIPIO')['RESIDENCIA'].to_dict()

                    except Exception as e:
                        print(f"Advertencia: Error al procesar hoja {hoja}: {str(e)}")
                        continue

                # Catálogo de edades personalizado
                edades = {}
                for i in range(0,18): edades[i]=0
                for i in range(18,30): edades[i]=1
                for i in range(30,40): edades[i]=2
                for i in range(40,50): edades[i]=3
                for i in range(50,60): edades[i]=4
                for i in range(60,121): edades[i]=5

                diccionario_alreves['Catálogo N AÑOS'] = edades
                edades_general = {
                    0: 'p_0a17',
                    1: 'p_18a29',
                    2: 'p_30a39',
                    3: 'p_40a49',
                    4: 'p_50a59',
                    5: 'p_60ymas'
                }
                diccionario_general['Catálogo N AÑOS'] = edades_general

            print("\nDiccionarios cargados exitosamente!")
            return diccionario_general, diccionario_alreves, descriptores

        except Exception as e:
            raise RuntimeError(f"Error al cargar diccionarios: {str(e)}")

In [10]:
class OptimizedNaiveBayesClassifier:
    def __init__(self, X, y):
        self.X, self.y = X, y
        self.N = len(X)
        self.feature_scores = {}  # Diccionario para almacenar scores por característica
        self.class_probs = {}     # Probabilidades por clase
        self.feature_values = []  # Valores únicos por característica
        self._precompute_scores()

    def _precompute_scores(self):
        """Precalcula todos los scores para cada valor de cada característica"""
        # Calcular probabilidades de clase
        unique, counts = np.unique(self.y, return_counts=True)
        self.class_probs = dict(zip(unique, counts/self.N))
        #print(self.class_probs)

        # Para cada característica
        for i in range(len(self.X[0])):
            feature_values = np.unique([x[i] for x in self.X])
            self.feature_values.append(feature_values)
            self.feature_scores[i] = {}

            # Para cada valor único de la característica
            for value in feature_values:
                # Calcular Nxc: casos con esta característica y clase positiva
                Nxc = sum(1 for x, cls in zip(self.X, self.y)
                         if x[i] == value and cls == 'yes')

                # Calcular Nx: total de casos con esta característica
                Nx = sum(1 for x in self.X if x[i] == value)

                # Suavizado Laplace
                a = 0.0005
                k = 16
                Nc = self.class_probs['yes'] * self.N

                # Calcular scores
                Pxc = (Nxc + a) / (Nc + k*a)
                Px_c = (Nx - Nxc + a) / (self.N - Nc + k*a)
                score = np.log(Pxc / Px_c)

                self.feature_scores[i][value] = score

    def calculate_total_score(self, features):
        """Calcula el score total sumando los scores precalculados"""
        total_score = np.log(self.class_probs['yes'] / self.class_probs['no'])

        for i, value in enumerate(features):
            total_score += self.feature_scores[i].get(value, 0)  # Usar get para manejar valores no vistos

        return total_score

    def batch_score(self, X):
        """Calcula scores para un conjunto de datos completo"""
        return [self.calculate_total_score(x) for x in X]

    def get_feature_contributions(self, features):
        """Obtiene la contribución porcentual de cada característica al score total"""
        total_score = np.log(self.class_probs['yes'] / self.class_probs['no'])
        contributions = []

        for i, value in enumerate(features):
            feature_score = self.feature_scores[i].get(value, 0)
            contributions.append((i, feature_score))
            total_score += feature_score

        # En get_feature_contributions, cambiar a:
        total_abs_score = sum(abs(score) for _, score in contributions)

        # Convertir a porcentajes
        if total_abs_score != 0:
            contributions = [(i, (abs(score)/total_abs_score)*100) for i, score in contributions]
        # Convertir a porcentajes
        #if total_score != 0:
        #    contributions = [(i, (score/total_score)*100) for i, score in contributions]

        return sorted(contributions, key=lambda x: abs(x[1])), total_score

In [11]:
from transformers import MarianTokenizer, MarianMTModel

class BioExplanationTranslator:
    def __init__(self):
        self.model_name = "Helsinki-NLP/opus-mt-en-es"
        self.tokenizer = MarianTokenizer.from_pretrained(self.model_name)
        self.model = MarianMTModel.from_pretrained(self.model_name)

    def translate(self, english_text):
        inputs = self.tokenizer(english_text, return_tensors="pt", truncation=True, max_length=512)
        translated = self.model.generate(**inputs)
        return self.tokenizer.decode(translated[0], skip_special_tokens=True)


##📄 6. Generación automática de reportes en PDF
Se implementa la clase PDFGenerator para construir un reporte visualmente estructurado, incluyendo gráficos y explicaciones generadas.

In [12]:
from fpdf import FPDF, XPos, YPos
import matplotlib.pyplot as plt
import seaborn as sns
import tempfile
import os
from datetime import datetime

class PDFGenerator:
    def __init__(self):
        self.pdf = FPDF()
        self.pdf.set_auto_page_break(auto=True, margin=15)
        self.temp_files = []
        self._setup_fonts()

    def _setup_fonts(self):
        # Configurar fuentes (evitar advertencias de sustitución)
        # Usando DejaVu para soporte Unicode
        self.pdf.add_font('DejaVu', '', '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf', uni=True)
        self.pdf.add_font('DejaVu', 'B', '/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf', uni=True)
        self.pdf.set_font('DejaVu', '', 12)  # Establecer DejaVu como fuente por defecto

    def add_page(self, title):
        self.pdf.add_page()
        self.pdf.set_font("DejaVu", "B", 16)
        self.pdf.cell(0, 10, title, new_x=XPos.LMARGIN, new_y=YPos.NEXT, align="C")
        self._add_space(10)  # Reemplazo de ln()

    def add_period_info(self, start_date, end_date):
        self.pdf.set_font('helvetica', '', 12)
        self.pdf.cell(0, 8, f"Periodo evaluado: {start_date} a {end_date}",
                     new_x=XPos.LMARGIN, new_y=YPos.NEXT)
        self._add_space(10)

    def add_disclaimer(self):
        self.pdf.set_font('helvetica', 'I', 10)
        self.pdf.set_text_color(150, 0, 0)  # Color rojo oscuro
        disclaimer_text = (
            "IMPORTANTE: Este reporte es solo para fines informativos y no sustituye "
            "el consejo médico profesional. Siempre consulte a un médico calificado "
            "para el diagnóstico y tratamiento."
        )
        self.pdf.multi_cell(0, 8, disclaimer_text)
        self.pdf.set_text_color(0, 0, 0)  # Volver a color negro
        self._add_space(15)

    def add_section(self, title, level=1):
        font_size = 14 if level == 1 else 12
        self.pdf.set_font("DejaVu", 'B', font_size)
        self.pdf.cell(0, 10, title, new_x=XPos.LMARGIN, new_y=YPos.NEXT)
        self._add_space(5)

    def add_text(self, text):
        self.pdf.set_font("DejaVu", size=12)
        self.pdf.multi_cell(0, 8, text)
        self._add_space(5)

    def add_bold_text(self, text):
        self.pdf.set_font("DejaVu", 'B', size=12)
        self.pdf.multi_cell(0, 8, text)
        self._add_space(5)

    def add_italic_text(self, text):
        self.add_text(text, 'I')

    def _add_space(self, height):
        """Método para añadir espacio vertical (reemplazo de ln())"""
        self.pdf.cell(0, height, "", new_x=XPos.LMARGIN, new_y=YPos.NEXT)

    def add_table(self, data, col_widths=None):
        """Agrega una tabla simple al PDF"""
        if col_widths is None:
            col_widths = [self.pdf.epw / len(data[0])] * len(data[0])

        self.pdf.set_font("helvetica", size=10)

        # Encabezados en negrita
        self.pdf.set_font("helvetica", "B", 10)
        for i, header in enumerate(data[0]):
            self.pdf.cell(col_widths[i], 10, header, border=1)
        self.pdf.ln()  # Usamos ln() directamente del objeto FPDF

        # Filas de datos
        self.pdf.set_font("helvetica", size=10)
        for row in data[1:]:
            for i, cell in enumerate(row):
                self.pdf.cell(col_widths[i], 10, str(cell), border=1)
            self.pdf.ln()

        self._add_space(10)

    def add_plot(self, fig):
        """Agrega una figura matplotlib al PDF"""
        temp_file = tempfile.NamedTemporaryFile(suffix=".png", delete=False)
        self.temp_files.append(temp_file.name)
        fig.savefig(temp_file.name, dpi=300, bbox_inches="tight")
        plt.close(fig)
        self.pdf.image(temp_file.name, x=10, w=190)
        self._add_space(10)

    def generate(self, filename):
        self.pdf.output(filename)
        # Limpiar archivos temporales
        for file in self.temp_files:
            try:
                os.unlink(file)
            except:
                pass

##🧪 7. Ejecución principal del proyecto
Se definen las rutas y se ejecutan los pasos de procesamiento de datos, clasificación y generación de reportes.

In [13]:
# -*- coding: utf-8 -*-
"""Proyecto_Version_Final_Mejorada.ipynb

Clasificador Naive Bayes para evaluación de riesgo clínico por COVID-19
con selección de variable objetivo y reportes mejorados
"""

class COVIDRiskAssessor:
    def __init__(self, covid_zip_path: str, dict_zip_path: str, explainer_model_path, recommender_model_path):
        """
        Inicializa el evaluador de riesgo COVID-19

        Args:
            covid_zip_path (str): Ruta al archivo ZIP con datos COVID-19
            dict_zip_path (str): Ruta al archivo ZIP con diccionarios
            qa_model (optional): Modelo de QA para evidencia científica
        """
        self.nbc = None
        self.data = None
        self.diccionario_general = {}
        self.diccionario_alreves = {}
        self.descriptores = {}
        self.target_variable = None
        self.feature_order = None
        self.explainer_pipe = pipeline("text-generation", model=explainer_model_path)
        self.recommender_pipe = pipeline("text-generation", model=recommender_model_path)

        # Inicializar cargadores
        self.data_loader = COVIDDataLoader(covid_zip_path)
        self.dict_loader = DictionaryLoader(dict_zip_path)

        # Cargar diccionarios al inicializar
        self.load_dictionaries()

                # Diccionario de mapeo ES -> EN
        self.comorbidity_translation = {
            "INTUBADO": "intubation",
            "NEUMONIA": "pneumonia",
            "EDAD": "age",
            "SEXO": "gender",
            "EMBARAZO": "pregnancy",
            "DIABETES": "diabetes",
            "EPOC": "COPD",
            "ASMA": "asthma",
            "INMUSUPR": "immune suppression",
            "HIPERTENSION": "hypertension",
            "CARDIOVASCULAR": "cardiovascular disease",
            "OBESIDAD": "obesity",
            "RENAL_CRONICA": "chronic renal disease",
            "TABAQUISMO": "smoking"
        }

        self.translator = BioExplanationTranslator()


    def generate_enhanced_report(self, evaluation_result, user_profile):
        """Genera un reporte completo con gráficos"""
        pdf = PDFGenerator()

        # Portada
        pdf.add_page(f"Reporte de Riesgo COVID-19 {evaluation_result['risk_level']}")
        pdf.add_text(f"Fecha de generación: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
        pdf.add_text(f"Variable objetivo: {self.target_variable}")
        pdf.add_period_info(start_date, end_date)


        # Resumen ejecutivo
        pdf.add_section("Resumen Ejecutivo")
        pdf.add_text(f"El paciente presenta un riesgo {evaluation_result['risk_level'].lower()} "
                    f"con una probabilidad estimada del {evaluation_result['probability']:.2f}%.")

        # Gráfico de probabilidad
        self._add_probability_chart(pdf, evaluation_result['probability'])

        # Factores de riesgo
        pdf.add_section("Factores de Riesgo Clave")
        self._add_risk_factors(pdf, evaluation_result['feature_contributions'])

        # Perfil del paciente
        pdf.add_section("Perfil Clínico del Paciente")
        self._add_patient_profile(pdf, user_profile)

        # Explicaciones médicas
        pdf.add_section("Explicaciones Médicas")
        explanations = self.generate_explanations_and_recommendations(user_profile)
        self._add_explanations(pdf, explanations)

        # Método técnico
        pdf.add_section("Metodología")
        pdf.add_text("Este reporte fue generado utilizando un modelo Naive Bayes optimizado "
                    "entrenado con datos oficiales de COVID-19 en México. Las probabilidades "
                    "se calculan comparando el perfil del paciente con la distribución de "
                    "riesgo en la población general.")
        pdf.add_disclaimer()


        # Generar PDF
        output_file = "reporte_riesgo_covid.pdf"
        pdf.generate(output_file)
        return output_file

    def _add_probability_chart(self, pdf, probability):
        """Agrega gráfico de probabilidad de riesgo"""
        fig, ax = plt.subplots(figsize=(10, 3))

        # Definir niveles de riesgo y colores
        risk_levels = ["Muy bajo", "Bajo", "Moderado", "Alto", "Muy alto"]
        thresholds = [0, 5, 15, 30, 50, 100]
        colors = ['#2ecc71', '#3498db', '#f1c40f', '#e67e22', '#e74c3c']

        # Crear gráfico de barras
        for i in range(len(thresholds)-1):
            ax.barh(0, thresholds[i+1]-thresholds[i], left=thresholds[i],
                   color=colors[i], alpha=0.7)

        # Marcar la posición del paciente
        ax.axvline(x=probability, color='black', linestyle='--', linewidth=2)
        ax.text(probability+1, 0.2, f'{probability:.1f}%',
               fontsize=12, va='center')

        # Configuraciones del gráfico
        ax.set_xlim(0, 100)
        ax.set_yticks([])
        ax.set_title('Nivel de Riesgo del Paciente', pad=20)
        ax.set_xlabel('Probabilidad (%)')

        # Leyenda
        patches = [plt.Rectangle((0,0),1,1, fc=colors[i])
                  for i in range(len(risk_levels))]
        ax.legend(patches, risk_levels, loc='upper center',
                 bbox_to_anchor=(0.5, -0.2), ncol=5)

        pdf.add_plot(fig)

    def _add_risk_factors(self, pdf, contributions):
        """Agrega gráfico de factores de riesgo"""
        # Preparar datos
        factors = [c['feature'] for c in contributions[:8]]
        values = [c['contribution'] for c in contributions[:8]]

        fig, ax = plt.subplots(figsize=(10, 5))
        sns.barplot(x=values, y=factors, palette='Blues_d', ax=ax)

        ax.set_title('Contribución de los Factores de Riesgo Principales')
        ax.set_xlabel('Contribución porcentual (%)')
        ax.set_ylabel('Factor')

        # Añadir valores a las barras
        for i, v in enumerate(values):
            ax.text(v + 0.5, i, f"{v:.1f}%", color='black', va='center')

        pdf.add_plot(fig)

        # Tabla detallada
        pdf.add_text("Detalle de factores de riesgo:")
        table_data = [["Factor", "Valor", "Contribución (%)"]]
        for c in contributions[:8]:
            table_data.append([c['feature'], str(c['value']), f"{c['contribution']:.1f}"])

        pdf.add_table(table_data, col_widths=[70, 70, 50])

    def _add_patient_profile(self, pdf, profile):
        """Agrega perfil del paciente"""
        table_data = [["Característica", "Valor"]]

        # Convertir valores a formato legible
        for key, value in profile.items():
            catalogo = 'Catálogo ' + self.descriptores.get(key, '')[10:]
            if catalogo in self.diccionario_general:
                readable_value = self.diccionario_general[catalogo].get(value, value)
            else:
                readable_value = value
            table_data.append([key, str(readable_value)])

        pdf.add_table(table_data, col_widths=[90, 100])

    def _add_explanations(self, pdf, explanations):
        """Agrega explicaciones médicas"""
        for exp in explanations:
            pdf.add_section(exp['comorbidity'], level=2)
            pdf.add_bold_text("Explicación:")
            pdf.add_text(exp['explanation'])
            pdf.add_bold_text("Recomendación:")
            pdf.add_text(exp['recommendation'])
            pdf._add_space(5)

    def limpiar_texto_generado(self, texto):
        texto = re.sub(r'(¿Le gustaría chatear.*?){2,}', '', texto)
        texto = re.sub(r'(Sea seguro\.? ?){2,}', 'Sea seguro. ', texto)
        texto = re.sub(r'(Sea positivo\.? ?){2,}', 'Sea positivo. ', texto)
        texto = re.sub(r'(La mejor opción.*?recomendación de salud\.){2,}', '', texto)
        texto = re.sub(r'https?:\/\/\S+', '', texto)
        return texto.strip()

    def generate_explanations_and_recommendations(self, clinical_profile):
        results = []

        for es_field, en_cond in self.comorbidity_translation.items():
            # Check if the value exists and convert it to a string before calling .upper()
            value = clinical_profile.get(es_field, "")
            if isinstance(value, str) and value.upper() == "SI":  # Check if value is a string and then apply upper()
                # Prompt for explanation technique
                explainer_prompt = (
                    f"<|startoftext|>\n"
                    f"[PROMPT]: How does {en_cond} affect COVID-19 outcomes?\n"
                    "[EXPLANATION]:"
                )

                #expl = self.explainer_pipe(explainer_prompt, max_length=300, do_sample=True, temperature=0.7)[0]['generated_text']

                # Generación con control
                expl_output = self.explainer_pipe(
                    explainer_prompt,
                    max_length=300,
                    do_sample=True,
                    temperature=0.8,
                    top_k=50,
                    top_p=0.95,
                    truncation=True,
                    early_stopping=True
                )[0]['generated_text']

                explanation_en = expl_output.split("[EXPLANATION]:")[-1].split("<|endoftext|>")[0].strip()

                # Traducir la explicación al español
                explanation_es = self.translator.translate(explanation_en)

                explanation_es = self.limpiar_texto_generado(explanation_es)


                # Prompt for practical recommendation
                recommender_prompt = (
                    f"<|startoftext|>\n"
                    f"[CONDITION]: {en_cond}\n"
                    f"[PROMPT]: Provide a clear and actionable health recommendation for a COVID-19 patient with this condition.\n"
                    f"[RECOMMENDATION]: For patients with {en_cond}, it is important to"
                )


                #rec = self.recommender_pipe(recommender_prompt, max_length=150, do_sample=True, temperature=0.7)[0]['generated_text']
                rec_output = self.recommender_pipe(
                    recommender_prompt,
                    max_length=150,
                    do_sample=True,
                    temperature=0.8,
                    top_k=50,
                    top_p=0.95,
                    truncation=True,
                    early_stopping=True
                )[0]['generated_text']

                recommendation_en = rec_output.split("[RECOMMENDATION]:")[-1].split("<|endoftext|>")[0].strip()

                # Traducir recomendación
                recommendation_es = self.translator.translate(recommendation_en)
                recommendation_es = self.limpiar_texto_generado(recommendation_es)

                results.append({
                    "comorbidity": es_field,
                    "explanation": explanation_es,
                    "recommendation": recommendation_es
                })

        return results

    def load_dictionaries(self):
        """Carga los diccionarios usando el DictionaryLoader"""
        self.diccionario_general, self.diccionario_alreves, self.descriptores = self.dict_loader.load_dictionaries()
        return self.diccionario_general, self.diccionario_alreves

    def load_data(self, start_date: str, end_date: str):
        """Carga datos usando el COVIDDataLoader"""
        # Columnas que necesitamos para el análisis
        columns = [
            'SEXO', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
            'FECHA_SINTOMAS', 'EDAD', 'EMBARAZO', 'DIABETES', 'EPOC',
            'ASMA', 'INMUSUPR', 'HIPERTENSION', 'CARDIOVASCULAR',
            'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO', 'FECHA_DEF',
            'INTUBADO', 'NEUMONIA', 'CLASIFICACION_FINAL', 'UCI'
        ]

        self.data = self.data_loader.load_data_by_date_range(start_date, end_date, columns)
        return self.data

    def preprocess_data(self, target_variable: str):
        """
        Preprocesa los datos para el modelo según la variable objetivo seleccionada

        Args:
            target_variable (str): Variable objetivo ('FALLECIMIENTO', 'HOSPITALIZADO',
                                  'INTUBADO', 'NEUMONIA', 'UCI')
        """
        self.target_variable = target_variable

        # Filtrar por clasificación (1,2,3,6 son casos confirmados)
        clasificacion = [1, 2, 3, 6]
        self.data = self.data[self.data['CLASIFICACION_FINAL'].isin(clasificacion)]

        # Crear columna RESIDENCIA combinando entidad y municipio
        self.data["RESIDENCIA"] = '[' + self.data['ENTIDAD_RES'].astype(str) + "," + \
                                 self.data["MUNICIPIO_RES"].astype(str) + ']'

        # Categorizar edades
        bins = [0, 17, 29, 39, 49, 59, 121]
        labels = [0, 1, 2, 3, 4, 5]
        self.data['EDAD'] = pd.cut(self.data['EDAD'], bins=bins, labels=labels, right=False)

        # Definir variable objetivo según selección
        if target_variable == 'FALLECIMIENTO':
            self.data['TARGET'] = np.where(self.data['FECHA_DEF'].notnull(), 'yes', 'no')
            self.feature_order = ['TIPO_PACIENTE', 'INTUBADO', 'NEUMONIA', 'MUNICIPIO_RES', 'EDAD',
                                'SEXO', 'EMBARAZO', 'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR',
                                'HIPERTENSION', 'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA',
                                'TABAQUISMO']

        elif target_variable == 'HOSPITALIZADO':
            self.data['TARGET'] = np.where(self.data['TIPO_PACIENTE'] == 2, 'yes', 'no')
            self.feature_order = ['MUNICIPIO_RES', 'EDAD', 'SEXO', 'EMBARAZO', 'DIABETES', 'EPOC',
                                'ASMA', 'INMUSUPR', 'HIPERTENSION', 'CARDIOVASCULAR', 'OBESIDAD',
                                'RENAL_CRONICA', 'TABAQUISMO']

        elif target_variable == 'INTUBADO':
            self.data['TARGET'] = np.where(self.data['INTUBADO'] == 1, 'yes', 'no')
            self.feature_order = ['TIPO_PACIENTE', 'NEUMONIA', 'MUNICIPIO_RES', 'EDAD', 'SEXO',
                                'EMBARAZO', 'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION',
                                'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO']

        elif target_variable == 'NEUMONIA':
            self.data['TARGET'] = np.where(self.data['NEUMONIA'] == 1, 'yes', 'no')
            self.feature_order = ['TIPO_PACIENTE', 'MUNICIPIO_RES', 'EDAD', 'SEXO', 'EMBARAZO',
                                'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION',
                                'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO']

        elif target_variable == 'UCI':
            self.data['TARGET'] = np.where(self.data['UCI'] == 1, 'yes', 'no')
            self.feature_order = ['MUNICIPIO_RES', 'EDAD', 'SEXO', 'EMBARAZO', 'DIABETES', 'EPOC',
                                'ASMA', 'INMUSUPR', 'HIPERTENSION', 'CARDIOVASCULAR', 'OBESIDAD',
                                'RENAL_CRONICA', 'TABAQUISMO']

        else:
            raise ValueError(f"Variable objetivo no válida: {target_variable}")

        # Seleccionar columnas finales
        features = self.feature_order + ['TARGET']
        self.data = self.data[features]

        return self.data

    def train_model(self):
        """Entrena el modelo optimizado"""
        y_train = self.data['TARGET'].values
        X_train = self.data.drop(columns=['TARGET']).values

        self.nbc = OptimizedNaiveBayesClassifier(X_train, y_train)
        return self.nbc

    def get_input_fields(self):
        """Devuelve los campos de entrada necesarios según la variable objetivo"""
        fields = []

        if 'TIPO_PACIENTE' in self.feature_order:
            fields.append(('TIPO_PACIENTE', "Tipo de paciente (AMBULATORIO/HOSPITALIZADO): "))

        if 'INTUBADO' in self.feature_order:
            fields.append(('INTUBADO', "¿Fue intubado? (SI/NO): "))

        if 'NEUMONIA' in self.feature_order:
            fields.append(('NEUMONIA', "¿Presenta neumonía? (SI/NO): "))

        if 'MUNICIPIO_RES' in self.feature_order:
            fields.append(('MUNICIPIO_RES', "Municipio de residencia: "))

        if 'EDAD' in self.feature_order:
            fields.append(('EDAD', "Edad: "))

        if 'SEXO' in self.feature_order:
            fields.append(('SEXO', "Sexo (HOMBRE/MUJER): "))

        if 'EMBARAZO' in self.feature_order:
            fields.append(('EMBARAZO', "¿Embarazo? (SI/NO/NO APLICA): "))

        # Campos de comorbilidades
        comorbilidades = [
            ('DIABETES', "¿Tiene diabetes? (SI/NO): "),
            ('EPOC', "¿Tiene EPOC? (SI/NO): "),
            ('ASMA', "¿Tiene asma? (SI/NO): "),
            ('INMUSUPR', "¿Tiene inmunosupresión? (SI/NO): "),
            ('HIPERTENSION', "¿Tiene hipertensión? (SI/NO): "),
            ('CARDIOVASCULAR', "¿Enfermedad cardiovascular? (SI/NO): "),
            ('OBESIDAD', "¿Tiene obesidad? (SI/NO): "),
            ('RENAL_CRONICA', "¿Enfermedad renal crónica? (SI/NO): "),
            ('TABAQUISMO', "¿Es fumador? (SI/NO): ")
        ]

        for comorb in comorbilidades:
            if comorb[0] in self.feature_order:
                fields.append(comorb)

        return fields

    def evaluate_risk(self, user_profile):
        """Evalúa el riesgo basado en el perfil del usuario"""
        # Convertir perfil a valores numéricos usando diccionarios
        numeric_profile = self._convert_profile(user_profile)

        # Obtener características en el orden correcto
        features = [numeric_profile[feat] for feat in self.feature_order]

        # Calcular score (suma de scores precalculados)
        score = self.nbc.calculate_total_score(features)

        # Obtener contribuciones de cada característica
        contributions, total_score = self.nbc.get_feature_contributions(features)

        # Mapear contribuciones a nombres de características
        feature_contributions = []
        for idx, contrib in contributions:
            feature_name = self.feature_order[idx]
            feature_value = user_profile.get(feature_name, 'N/A')

            # Obtener descripción legible del valor
            catalogo = 'Catálogo ' + self.descriptores.get(feature_name, '')[10:]
            desc_value = self.diccionario_general.get(catalogo, {}).get(numeric_profile[feature_name], feature_value)

            feature_contributions.append({
                'feature': feature_name,
                'value': desc_value,
                'contribution': abs(contrib)
            })

        # Ordenar por contribución descendente
        feature_contributions = sorted(feature_contributions, key=lambda x: x['contribution'], reverse=True)

        # Obtener percentil y probabilidad
        percentile, probability = self._get_percentile_and_probability(score)

        return {
            'score': score,
            'percentile': percentile,
            'probability': probability*100,
            'risk_level': self._interpret_risk(probability*100),
            'feature_contributions': feature_contributions
        }

    def _convert_profile(self, profile):
        """Convierte perfil de usuario a valores numéricos usando diccionarios"""
        numeric_profile = {}
        """
        for clave, valor in profile.items():

            if clave == 'EDAD':
                # Manejar edad especial (ya viene como número)
                numeric_profile[clave] = valor
                continue

            catalogo = 'Catálogo ' + self.descriptores.get(clave, '')[10:]
            if catalogo in self.diccionario_alreves:
                numeric_profile[clave] = self.diccionario_alreves[catalogo].get(str(valor).upper(), 0)
            else:
                print(f"Advertencia: No se encontró catálogo para {clave}")
                numeric_profile[clave] = 1 if str(valor).upper() == 'SI' else 0
        """
        for clave, valor in profile.items():
            catalogo = 'Catálogo ' + self.descriptores[clave][10:]
            numeric_profile[clave] = self.diccionario_alreves[catalogo][valor]

        return numeric_profile

    def _get_percentile_and_probability(self, score, n_bins=20):
        """Calcula percentil y probabilidad usando scores precalculados"""
        if not hasattr(self, '_score_distribution'):
            # Calcular scores para todos los datos una sola vez
            X = self.data.drop(columns=['TARGET']).values
            self._scores = self.nbc.batch_score(X)
            self._classes = self.data['TARGET'].values

            # Crear bins
            self._score_distribution = pd.DataFrame({
                'score': self._scores,
                'class': self._classes
            })
            self._score_distribution['bin'] = pd.qcut(
                self._score_distribution['score'],
                q=n_bins,
                labels=False
            )

            # Calcular estadísticas por bin
            self._bin_stats = self._score_distribution.groupby('bin').agg(
                min_score=('score', 'min'),
                max_score=('score', 'max'),
                probability=('class', lambda x: (x == 'yes').mean())
            ).reset_index()

        # Encontrar el bin correspondiente
        bin_info = self._bin_stats[
            (self._bin_stats['min_score'] <= score) &
            (self._bin_stats['max_score'] >= score)
        ].iloc[0]

        return bin_info['bin'], bin_info['probability']

    def _interpret_risk(self, probability):
        """Interpreta la probabilidad como nivel de riesgo"""
        if probability < 5:
            return "Muy bajo"
        elif probability < 15:
            return "Bajo"
        elif probability < 30:
            return "Moderado"
        elif probability < 50:
            return "Alto"
        else:
            return "Muy alto"

    def generate_report(self, evaluation_result):
        """Genera un reporte detallado de la evaluación de riesgo"""
        report = f"""
## Evaluación de Riesgo COVID-19 - {evaluation_result['risk_level']}
**Variable objetivo:** {self.target_variable}
**Probabilidad estimada:** {evaluation_result['probability']:.2f}%

### Factores Clave que Afectan su Riesgo:
"""
        for contrib in evaluation_result['feature_contributions']:  # Mostrar solo los 8 principales
            report += f"- **{contrib['feature']}**: {contrib['value']} (contribución: {contrib['contribution']:.1f}%)\n"

        report += f"""
### Detalles Técnicos:
- **Score de riesgo calculado:** {evaluation_result['score']:.2f}
- **Percentil de riesgo:** {evaluation_result['percentile']+1}/20
- **Metodología:** Modelo Naive Bayes optimizado con suavizado Laplace
"""

        return report

In [14]:
if __name__ == "__main__":
    # Configurar rutas (pueden ser ingresadas por el usuario)
    COVID_ZIP_PATH = "/content/drive/MyDrive/COVID19MEXICO.zip" # path de COVID19MEXICO.zip
    DICT_ZIP_PATH = "/content/drive/MyDrive/diccionario_datos_abiertos_corregido.zip" # path de diccionario_datos_abiertos_corregido.zip

    # Crear evaluador con rutas personalizadas
    evaluator = COVIDRiskAssessor(
        covid_zip_path=COVID_ZIP_PATH,
        dict_zip_path=DICT_ZIP_PATH,
        explainer_model_path="/content/biogpt_explainer_model_final", # path del modelo biogpt_explainer_model_final
        recommender_model_path="/content/clean_biogpt_comorbidity_model_final" #path del modelo clean_biogpt_comorbidity_model_final
    )

    # Cargar datos para un período específico (pueden ser parámetros de entrada)
    # Pedir fechas al usuario
    print("Ingrese el periodo de análisis con fechas en formato AAAA-MM-DD.")
    start_date = input("📅 Fecha de inicio: ")
    end_date = input("📅 Fecha de fin: ")


    print(f"\nCargando datos desde {start_date} hasta {end_date}...")
    evaluator.load_data(start_date, end_date)

    if evaluator.data.empty:
        print("\nNo se encontraron datos para el rango especificado")
    else:
        # Seleccionar variable objetivo
        print("\nSeleccione la variable objetivo para evaluar el riesgo:")
        print("1. FALLECIMIENTO")
        print("2. HOSPITALIZADO")
        print("3. INTUBADO")
        print("4. NEUMONIA")
        print("5. UCI")

        target_choice = input("Ingrese el número de opción (1-5): ")
        target_map = {
            '1': 'FALLECIMIENTO',
            '2': 'HOSPITALIZADO',
            '3': 'INTUBADO',
            '4': 'NEUMONIA',
            '5': 'UCI'
        }

        target_variable = target_map.get(target_choice, 'FALLECIMIENTO')

        # Preprocesar datos según variable objetivo
        print(f"\nPreprocesando datos para variable objetivo: {target_variable}...")
        evaluator.preprocess_data(target_variable)

        # Entrenar modelo
        print("\nEntrenando modelo...")
        evaluator.train_model()

        # Obtener campos de entrada según variable objetivo
        input_fields = evaluator.get_input_fields()

        # Crear perfil de usuario
        user_profile = {}
        print("\nIngrese los datos del paciente:")
        for field, prompt in input_fields:
            if field == 'EDAD':
                user_profile[field] = int(input(prompt))
            else:
                user_profile[field] = input(prompt).upper()

        #print("\nPerfil de usuario:")
        #print(user_profile)

        # Evaluar riesgo
        print("\nEvaluando riesgo...")
        resultado = evaluator.evaluate_risk(user_profile)
        #print(resultado)

        # Generar y mostrar reporte
        report = evaluator.generate_report(resultado)
        print(report)

        resultados = evaluator.generate_explanations_and_recommendations(user_profile)

        for r in resultados:
            print(f"🧬 *{r['comorbidity']}*")
            print(f"🩺 Explicación médica: {r['explanation']}")
            print(f"💡 Recomendación práctica: {r['recommendation']}")
            print("—" * 40)

    # Generar y mostrar reporte
    print("\nGenerando reporte en PDF...")
    pdf_file = evaluator.generate_enhanced_report(resultado, user_profile)
    print(f"✅ Reporte generado exitosamente: {pdf_file}")

    # Opcional: mostrar en Colab
    try:
        from google.colab import files
        files.download(pdf_file)
    except:
        pass


Device set to use cuda:0
Device set to use cuda:0



Diccionarios cargados exitosamente!


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

...

🧬 *DIABETES*
🩺 Explicación médica: ANTECEDENTES: La relación entre la diabetes y los pacientes con COVID-19 no está bien comprendida. MÉTODOS: Este estudio investigó el impacto de la diabetes en los pacientes con COVID-19 en un estudio multicéntrico de cohorte prospectivo. Los sujetos del estudio incluyeron pacientes con COVID-19 confirmado (n = 1032). Los pacientes fueron categorizados en dos grupos: aquellos con y sin diabetes. Las características clínicas, los hallazgos de laboratorio, las estrategias de tratamiento y los resultados fueron comparados entre los dos grupos. RESULTADOS: Diabetes se encontró en 130 (13,0%) de los pacientes con COVID-19. En comparación con los pacientes no diabéticos, los pacientes diabéticos fueron más frecuentemente varones (p = 0,004), fumadores (p < 0,001) y no obesos (p < 0,001). La mediana de edad de los pacientes diabéticos fue significativamente mayor que la de los pacientes no diabéticos (p < 0,001). En comparación con los pacientes no diabético

<ipython-input-12-c50b946d069e>:18: DeprecationWarning: "uni" parameter is deprecated since v2.5.1, unused and will soon be removed
  self.pdf.add_font('DejaVu', '', '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf', uni=True)
<ipython-input-12-c50b946d069e>:19: DeprecationWarning: "uni" parameter is deprecated since v2.5.1, unused and will soon be removed
  self.pdf.add_font('DejaVu', 'B', '/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf', uni=True)
<ipython-input-13-e2e866e40c71>:142: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=values, y=factors, palette='Blues_d', ax=ax)
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:679: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1`

✅ Reporte generado exitosamente: reporte_riesgo_covid.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>